In [7]:
# Importando o arquivo direto da base do TSE
from urllib.request import urlretrieve
import zipfile

#urlretrieve ('http://agencia.tse.jus.br/estatistica/sead/eleicoes/eleicoes2016/buweb/bweb_1t_RJ_04102016185804.zip','bweb.zip')

zip_ref = zipfile.ZipFile('bweb.zip', 'r')
zip_ref.extractall('.')
zip_ref.close()

In [3]:
# Totaliza os votos da cidade e os votos específicos da Marielle

import csv
import collections

campos = ['data','hora','pleito','eleicao','uf','cargo','descricao_cargo','zona','secao','local',
          'num_partido','partido','cod_municipio','municipio','dt_bu','aptos','faltosos','comparecimento',
          'tipo_eleicao', 'tipo_urna', 'descr_urna','num_candidato','nome_candidato','votos_candidato',]
tot_registros = 0
tot_votos = 0

# Objetivo de totalizar todas as zonas: para saber exatamente quantas zonas tiveram votos já que a lista
# do TRE está desatualizada
zonas_total = collections.Counter()
secoes_total = collections.Counter()

zonas_marielle = collections.Counter()
secoes_marielle = collections.Counter()

# Total de Candidatos concorrendo
candidatos = collections.Counter()

num_candidato = '50777' # Número da Marielle
municipio = '60011'  # Rio de Janeiro

tot_votos_geral = 0
arquivo = open('bweb_1t_RJ_04102016185804.txt', encoding='iso-8859-1')
for registro in csv.DictReader(arquivo, fieldnames=campos, delimiter=';'):

    # considerar apenas os votos para vereador, do municipio do Rio, descartando os nulos e brancos
    if registro['cod_municipio'] == municipio and registro['cargo'] == '13' and registro['num_partido'] != '99' and len(registro['num_candidato']) == 5: ### Legenda não entra
        num_votos = int(registro['votos_candidato'])
        chave = registro['zona']+'/'+registro['secao']

        zonas_total[ registro['zona'] ] += num_votos
        secoes_total[ chave ] += num_votos
        tot_votos_geral += num_votos
        
        candidatos[ registro['num_candidato'] ] += num_votos

        if registro['num_candidato'] == num_candidato:
            zonas_marielle[ registro['zona'] ] += num_votos
            secoes_marielle[chave] += num_votos
            tot_votos += num_votos
   
print('Total Geral')
print('Total de Votos: %d' % tot_votos_geral)
print('Total de Candidatos: %d' % len(candidatos) )
print('Total de Zonas: %d' % len(zonas_total) )
print('Total de Seções: %d' % len(secoes_total) )
print('')

print('Total Marielle')
print('Total de Votos: %d' % tot_votos)
print('Total de Zonas: %d' % len(zonas_marielle) )
print('Total de Seções: %d' % len(secoes_marielle))

Total Geral
Total de Votos: 2689954
Total de Candidatos: 1590
Total de Zonas: 97
Total de Seções: 11609

Total Marielle
Total de Votos: 46502
Total de Zonas: 97
Total de Seções: 9472


In [9]:
# Média dos votos dos candidatos
media = tot_votos_geral / len(candidatos)
print( media )

# Total de Candidatos com mais votos que a média
acima_media = 0
for candidato, votos in candidatos.most_common():
    if votos > media:
        acima_media += 1
    else:
        break
print('Candidatos com votos acima da média: %d' % acima_media )

1691.794968553459
Candidatos com votos acima da média: 281


In [10]:
checksum = 0
for zona, votos in zonas_marielle.most_common():
    print('Zona %s tem %s votos' % (zona, votos))
    checksum += votos
    
print(checksum)
print('Média: %d' % ( checksum / len(zonas_marielle)) )

Zona 16 tem 2237 votos
Zona 4 tem 1926 votos
Zona 7 tem 1837 votos
Zona 161 tem 1688 votos
Zona 9 tem 1333 votos
Zona 3 tem 1286 votos
Zona 166 tem 1170 votos
Zona 17 tem 1160 votos
Zona 163 tem 1113 votos
Zona 211 tem 1107 votos
Zona 170 tem 1090 votos
Zona 252 tem 1011 votos
Zona 212 tem 996 votos
Zona 119 tem 949 votos
Zona 164 tem 907 votos
Zona 173 tem 796 votos
Zona 19 tem 785 votos
Zona 179 tem 731 votos
Zona 5 tem 722 votos
Zona 13 tem 707 votos
Zona 192 tem 694 votos
Zona 180 tem 682 votos
Zona 6 tem 680 votos
Zona 171 tem 666 votos
Zona 206 tem 638 votos
Zona 165 tem 632 votos
Zona 214 tem 626 votos
Zona 228 tem 615 votos
Zona 205 tem 609 votos
Zona 182 tem 605 votos
Zona 216 tem 604 votos
Zona 229 tem 571 votos
Zona 20 tem 564 votos
Zona 215 tem 521 votos
Zona 18 tem 518 votos
Zona 213 tem 508 votos
Zona 191 tem 490 votos
Zona 210 tem 474 votos
Zona 204 tem 463 votos
Zona 160 tem 459 votos
Zona 169 tem 433 votos
Zona 185 tem 420 votos
Zona 11 tem 400 votos
Zona 190 tem 393 v

In [28]:
# E como agrupar por grandes regiões: NORTE, SUL, OESTE, CENTRO, LEOPOLDINA e BARRA

area = {}
with open('secao_bairro.csv') as arquivo_zona:
    for registro in csv.DictReader(arquivo_zona, delimiter=','):
        chave = registro['zona']+'/'+registro['secao']
        area[ chave ] = [ registro['bairro'], registro['regiao'], registro['eleitores'] ]
        
print(len(area))

11804


In [34]:
# Agrupando as votações da Marielle por bairro (pois um bairro tem várias seções eleitorais)
total_bairro = collections.Counter()
total_regiao = collections.Counter()
for secao, votos in secoes_marielle.most_common():
    if secao in area:
        total_bairro[ area[secao][0] ] += votos
        total_regiao[ area[secao][1] ] += votos
    else:
        total_bairro[ 'ND' ] += votos
        print( zona )

for descricao, votos in total_bairro.most_common():
    print('%s: %s votos' % (descricao, votos))

TIJUCA: 3637 votos
COPACABANA: 2742 votos
BOTAFOGO: 2115 votos
LARANJEIRAS: 1900 votos
BONSUCESSO: 1579 votos
BARRA: 1325 votos
FLAMENGO: 1157 votos
VILA ISABEL: 1060 votos
LEBLON: 1027 votos
MEIER: 931 votos
IPANEMA: 884 votos
TAQUARA: 848 votos
CATETE: 844 votos
GÁVEA: 828 votos
JARDIM BOTÂNICO: 816 votos
HUMAITA: 799 votos
CAMPO GRANDE: 781 votos
GRAJAÚ: 752 votos
FREGUESIA JPA: 707 votos
SANTA TERESA: 687 votos
ANDARAÍ: 586 votos
RAMOS: 567 votos
REALENGO: 522 votos
RECREIO: 509 votos
CENTRO: 508 votos
JARDIM GUANABARA: 452 votos
PENHA: 446 votos
OLARIA: 443 votos
MARACANÃ: 434 votos
BANGU: 428 votos
RIO COMPRIDO: 427 votos
CACHAMBI: 424 votos
LAGOA: 388 votos
ENGENHO NOVO: 388 votos
IRAJÁ: 366 votos
VILA DA PENHA: 361 votos
ENGENHO DE DENTRO: 360 votos
COSME VELHO: 337 votos
SÃO CONRADO: 323 votos
VILA VALQUEIRE: 317 votos
PRAÇA SECA: 316 votos
URCA: 311 votos
PECHINCHA: 301 votos
TODOS OS SANTOS: 298 votos
HIGIENÓPOLIS: 298 votos
LEME: 296 votos
SÃO CRISTÓVÃO: 279 votos
PIEDADE: 

In [36]:
# Mapa de votos de Marielle por região

print('Regiões')
for descricao, votos in total_regiao.most_common():
    print('%s: %s votos' % (descricao, votos))

Regiões
ZONA SUL: 15072 votos
ZONA NORTE: 8514 votos
GRANDE TIJUCA: 7084 votos
ZONA OESTE: 3994 votos
LEOPOLDINA: 3214 votos
JACAREPAGUA: 3004 votos
CENTRO: 2094 votos
BARRA: 1963 votos
ILHA DO GOV: 1563 votos


In [30]:
# Quantos votantes existem na ROCINHA ?
total_bairro = 0
for registro in list(area.values()):
    if registro[0] == 'ROCINHA':
        total_bairro += int(registro[2])
        
print(total_bairro)

37026


In [43]:
# Total de Votantes por região e percentual de votos da Marielle
votantes_regiao = collections.Counter()

for registro in list(area.values()):
    votantes_regiao[ registro[1] ] += int(registro[2])
        
for regiao, votos in votantes_regiao.most_common():
    if votos != 0:
        perc = total_regiao[regiao] / votos * 100
        print('%s: %d (%f%%)' % (regiao,votos, perc))

ZONA NORTE: 1373980 (0.619660%)
ZONA OESTE: 1350657 (0.295708%)
ZONA SUL: 575978 (2.616767%)
JACAREPAGUA: 393852 (0.762723%)
GRANDE TIJUCA: 380523 (1.861648%)
LEOPOLDINA: 328038 (0.979765%)
CENTRO: 173581 (1.206353%)
ILHA DO GOV: 170992 (0.914078%)
BARRA: 150444 (1.304804%)


In [18]:
# Será que a distribuição de votos de outros candidatos se comportam da mesma forma ?

# A candidata ROSA FERNANDES tem uma capilaridade bem menor em número de seções!

import csv
import collections

zonas_candidato = collections.Counter()
secoes_candidato = collections.Counter()

# Número do candidato X
tot_votos = 0
num_candidato = '15150' # Rosa Fernandes
municipio = '60011'  # Rio de Janeiro

arquivo = open('bweb_1t_RJ_04102016185804.txt', encoding='iso-8859-1')
for registro in csv.DictReader(arquivo, fieldnames=campos, delimiter=';'):

    if registro['cod_municipio'] == municipio and registro['cargo'] == '13': 
        if registro['num_candidato'] == num_candidato:
            num_votos = int(registro['votos_candidato'])
            zonas_candidato[ registro['zona'] ] += num_votos
            chave = registro['zona']+'/'+registro['secao']
            secoes_candidato[chave] += num_votos
            tot_votos += num_votos
    
print('Estatística do candidato %s' % num_candidato)
print('Número de Votos %s' % tot_votos )
print('Total de Zonas %s' % len(zonas_candidato))
print('Total de Seções: %d' % len(secoes_candidato))

Estatística do candidato 15150
Número de Votos 57868
Total de Zonas 97
Total de Seções: 7043


In [20]:
# Qual o candidato que teve mais votos nos bairros de RAMOS, BONSUCESSO e MARE ?

import csv
import collections

candidatos = collections.Counter()

tot_votos_validos = 0
municipio = '60011'  # Rio de Janeiro

# Monta a lista de zonas/seções que estão localizadas na Maré
secoes_bairro = collections.Counter()
for secao in area:
    registro = area.get(secao)
#    if registro[1] == 'LEOPOLDINA':
    if registro[0] in ('BONSUCESSO','RAMOS','MARÉ'):
        secoes_bairro[ secao ] += int(registro[2])
        
print('%d seções' % len(secoes_bairro))

with open('bweb_1t_RJ_04102016185804.txt', encoding='iso-8859-1') as arquivo:
    for registro in csv.DictReader(arquivo, fieldnames=campos, delimiter=';'):

        if registro['cod_municipio'] == municipio and registro['cargo'] == '13': 
            chave = registro['zona']+'/'+registro['secao']
            if chave in secoes_bairro:
                candidatos[ registro['num_candidato']] += int(registro['votos_candidato'])
    
# Mostra os candidatos mais votados da Maré
for candidato, votos in candidatos.most_common():
    print('Candidato %s: %d votos' % (candidato, votos))
    
# Resultado apenas da Maré: 14356 - ALTAIR VILLELA DE SOUZA (PTB)
# Reparem que a concentração dos votos dele ocorre apenas na Maré!
# https://eleicoesepolitica.net/vereador2016/vereador/RJ/60011/14356

# Mas em Bonsucesso, Marielle já foi a 7a mais votada!
# E o primeiro lugar também tem os votos concentrados no bairro!
# https://eleicoesepolitica.net/vereador2016/vereador/RJ/60011/22555

# Por fim, se fizermos a busca nos 3 bairros próximos, o candidato que recebeu mais votos foi 
# HIDELBRANDO RODRIGUES com 8700 votos.
# https://eleicoesepolitica.net/vereador2016/vereador/RJ/60011/22555
# Este candidato também teve baixíssima capilaridade.

372 seções
Candidato 96: 14220 votos
Candidato 95: 10828 votos
Candidato 22555: 8723 votos
Candidato 45245: 6785 votos
Candidato 15100: 4701 votos
Candidato 10: 3080 votos
Candidato 15105: 2929 votos
Candidato 25551: 2546 votos
Candidato 25181: 2502 votos
Candidato 50777: 2196 votos
Candidato 20120: 2133 votos
Candidato 11404: 1819 votos
Candidato 55855: 1636 votos
Candidato 10123: 1493 votos
Candidato 50123: 1441 votos
Candidato 15150: 1413 votos
Candidato 10567: 1387 votos
Candidato 14356: 1353 votos
Candidato 25500: 1267 votos
Candidato 12333: 1226 votos
Candidato 25025: 1204 votos
Candidato 15: 1138 votos
Candidato 20200: 1093 votos
Candidato 10999: 1021 votos
Candidato 77044: 1014 votos
Candidato 44400: 927 votos
Candidato 50: 894 votos
Candidato 12603: 848 votos
Candidato 15777: 749 votos
Candidato 31330: 746 votos
Candidato 12200: 563 votos
Candidato 11111: 529 votos
Candidato 17420: 522 votos
Candidato 45: 514 votos
Candidato 15002: 483 votos
Candidato 19940: 478 votos
Candidat